### Generate sample paths of a McKean-Vlasov SDE using PyTorch

Consider McKean-Vlasov SDE of the following form:

$$ 
\textrm{d}X_t = (\alpha \, X_t + \beta \, \mathbb{E}[X_t]) \textrm{d} t + \sigma \, \textrm{d} W_t, \quad X_0 = x_0.
$$

We approximate the solution $X_t, t \in [0,T]$ by using the following 

$$ 
\textrm{d}X_t = \bigl(\alpha \, X_t + \beta \, \sum^M_{i=1} \gamma_i \mathbf{1}_{[t_{i-1},t_i[}(t) \bigr) \textrm{d} t + \sigma \, \textrm{d} W_t, \quad X_0 = x_0,
$$

where we approximate $\mathbb{E}[X_t]$ for $t \in [t_{i-1},t_i[$ by a single constant $\gamma_i.$ Set $t_0 = 0$ and $t_M = T.$


In [36]:
import torch
import random
from torch import optim, nn
import math
import matplotlib.pyplot as plt
from torchviz import make_dot

In [37]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [42]:
class MKVSDElinear_approx(torch.nn.Module):
    def __init__(self, gamma):
        super().__init__()
        # wrap gamma with nn.Parameter to compute gradients with respect to it using autograd
        self.gamma = nn.Parameter(gamma)
    def getpath(self, x0, alpha, beta, sigma, M, N, dt, dW): 
        X = x0 * torch.ones(N, M + 1)
        X[:, 1:M+1] = X[:, 0:M] * (1 + alpha * dt) + beta * dt * self.gamma + sigma * dW
        return X
    def getgradpath(self, x0, alpha, beta, sigma, M, N, dt):
        xi = torch.zeros(N, M, M + 1) # N paths with M gradient values based on size of gamma, and M + 1 values
        for i in range(M):
            xi[:,:,i+1] += xi[:,:,i] * (1 + alpha * dt) + beta * dt
        return xi

# seed initialise
torch.manual_seed(42)

# model parameters
M = 5  # number of discretisation steps
N = 10 # number of sample path
x0 = 1
sigma = 1
alpha = - 0.5
beta = 0.3
T = 0.2
dt = T / M

# simulate Brownian increments
dW = dt**0.5 * torch.randn(size=(N, M))
# generate random values for gamma initialisation
gamma = torch.randn(M, requires_grad = True, dtype=torch.float, device=device)

MKVSDE_Xapprox = MKVSDElinear_approx(gamma).to(device)
MKVSDE_Xapproxpath = MKVSDE_Xapprox.getpath(x0, alpha, beta, sigma, M, N, dt, dW)
MKVSDE_Xapprox_gradpath = MKVSDE_Xapprox.getgradpath(x0, alpha, beta, sigma, M, N, dt) 

print(MKVSDE_Xapproxpath.size())
print(MKVSDE_Xapprox_gradpath[1,:,:])
# MKVSDE_Xapproxpath.backward(torch.ones_like(gamma))


torch.Size([10, 6])
tensor([[0.0000, 0.0120, 0.0238, 0.0353, 0.0466, 0.0576],
        [0.0000, 0.0120, 0.0238, 0.0353, 0.0466, 0.0576],
        [0.0000, 0.0120, 0.0238, 0.0353, 0.0466, 0.0576],
        [0.0000, 0.0120, 0.0238, 0.0353, 0.0466, 0.0576],
        [0.0000, 0.0120, 0.0238, 0.0353, 0.0466, 0.0576]])


In [52]:
x = torch.Tensor([2, 1, 2, 3, 4, 5, 6, 7])
x[1:4] = torch.cumsum(x[0:3], dim=0)
print(x)

tensor([2., 2., 3., 5., 4., 5., 6., 7.])
